In [1]:
# Step 1: Import libraries
import os
from dotenv import load_dotenv

# Step 2: Load .env file
load_dotenv()

# Step 3: Fetch the API key
openai_key = os.getenv("OPENAI_API_KEY")

# Step 4: Print to confirm
print("API Key loaded? ", bool(openai_key))


API Key loaded?  True


In [3]:
import re   # re = regular expressions (used for pattern matching)

def safe_calculator(expression: str) -> str:
    """
    A tiny safe calculator for +, -, *, /, ()
    Example: "879*11" -> "9669"
    """
    # Step 1: Check if expression only has digits, spaces, and math symbols
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\)\s]+", expression):
        return "Invalid expression. Allowed: digits and + - * / ( )"
    
    try:
        # Step 2: Evaluate math safely (disable Python builtins)
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)  # convert result into a string
    except Exception as e:
        # Step 3: If error occurs (like bad input), show it
        return f"Error: {e}"


In [ ]:
print(safe_calculator("879*111"))      # should return "9669"
print(safe_calculator("(100+25)/5"))  # should return "25.0"
print(safe_calculator("abc+1"))       # should say "Invalid expression"


9669
25.0
Invalid expression. Allowed: digits and + - * / ( )


In [15]:
def translate_text(text: str, target_language: str = "Hindi") -> str:
    """
    Translate text into the target language using OpenAI.
    """
    system = SystemMessage(content="You are a precise translator. Only return the translation.")
    human = HumanMessage(content=f"Translate to {target_language}: {text}")
    response = translator_llm.invoke([system, human])
    return response.content.strip()


In [16]:
print(translate_text("Good morning, team", "Hindi"))
print(translate_text("How are you?", "Hindi"))
print(translate_text("नमस्ते दोस्तों", "English"))


सुप्रभात, टीम
आप कैसे हैं?
Hello friends


In [10]:
print(translate_text("Good morning, team", "Hindi"))
print(translate_text("How are you?", "Hindi"))
print(translate_text("नमस्ते दोस्तों", "English"))


सुप्रभात, टीम
आप कैसे हैं?
Hello friends


In [11]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Use a reasoning-friendly model for planning (pass API key explicitly)
planner_llm = ChatOpenAI(
    model="gpt-4.1",        # you can also try "gpt-4o-mini" for cheaper tests
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY")
)


In [12]:
SUPERVISOR_SYSTEM = SystemMessage(content=
    "You are a planner that breaks a user's request into a JSON 'plan'. "
    "Allowed steps:\n"
    "1) calculate: needs {expression}\n"
    "2) translate: needs {text, target_language}\n\n"
    "Rules:\n"
    "- Output ONLY valid JSON with a top-level key 'plan' (list of steps).\n"
    "- No explanations. JSON only.\n"
    "- If a step needs the previous step's result, set text to '$LAST_RESULT'.\n"
    "- Keep the plan minimal and sufficient."
)


In [13]:
import json
import re

def make_plan(user_input: str) -> dict:
    """
    Ask the planner LLM to output a JSON plan.
    If JSON parsing fails, use a safe fallback plan.
    """
    # Give the model 2 simple examples to anchor the format
    examples = (
        "Example 1: '879*11'\n"
        '{"plan":[{"step":"calculate","expression":"879*11"}]}\n\n'
        "Example 2: 'Translate the answer of 879*11 to Hindi'\n"
        '{"plan":[{"step":"calculate","expression":"879*11"},'
        '{"step":"translate","text":"$LAST_RESULT","target_language":"Hindi"}]}'
    )

    # Build the user message
    msg = HumanMessage(content=f"{examples}\n\nUser request: {user_input}\nReturn JSON only.")

    # Ask the planner
    resp = planner_llm.invoke([SUPERVISOR_SYSTEM, msg])

    # Full raw text (should be JSON)
    raw = resp.content.strip()
    print("DEBUG - Raw planner output:\n", raw)  # helpful while learning

    # Try to parse JSON
    try:
        plan = json.loads(raw)
        # minimal sanity check
        if isinstance(plan, dict) and "plan" in plan and isinstance(plan["plan"], list):
            return plan
    except json.JSONDecodeError:
        pass

    # Fallback so notebook never crashes
    has_math = bool(re.search(r"[0-9]\s*[\+\-\*\/\)]|[\(]\s*[0-9]", user_input))
    wants_translate = "translate" in user_input.lower()
    if has_math and wants_translate:
        return {"plan":[
            {"step":"calculate","expression":user_input},
            {"step":"translate","text":"$LAST_RESULT","target_language":"Hindi"}
        ]}
    elif has_math:
        return {"plan":[{"step":"calculate","expression":user_input}]}
    elif wants_translate:
        return {"plan":[{"step":"translate","text":user_input,"target_language":"Hindi"}]}
    else:
        # with only two agents, default to translate whole input to Hindi
        return {"plan":[{"step":"translate","text":user_input,"target_language":"Hindi"}]}

def run_plan(user_input: str) -> str:
    """
    1) Get a JSON plan via make_plan(...)
    2) Execute each step in order (calculate / translate)
    3) Return the final output
    """
    plan = make_plan(user_input)
    print("\nDEBUG - Parsed plan:")
    print(json.dumps(plan, indent=2))

    last_result = None

    for i, step in enumerate(plan.get("plan", []), start=1):
        action = step.get("step", "")

        if action == "calculate":
            expr = step.get("expression", "")
            print(f"\nSTEP {i}: CALCULATE  -> {expr}")
            last_result = safe_calculator(expr)
            print("Result:", last_result)

        elif action == "translate":
            text = step.get("text", "")
            if text == "$LAST_RESULT":
                text = last_result or ""
            lang = step.get("target_language", "Hindi")
            print(f"\nSTEP {i}: TRANSLATE -> '{text}' to {lang}")
            last_result = translate_text(text, lang)
            print("Translation:", last_result)

        else:
            print(f"\nSTEP {i}: Unknown step:", action)
            last_result = "Unknown step produced by planner."

    print("\nFINAL:", last_result)
    return last_result or ""


In [14]:
print("\n--- Test 1: Pure Math ---")
run_plan("879*11")

print("\n--- Test 2: Translate only ---")
run_plan("Translate 'Good morning, team' to Hindi")

print("\n--- Test 3: Combined (calculate → translate) ---")
run_plan("Translate the answer of 879*11 into Hindi")



--- Test 1: Pure Math ---
DEBUG - Raw planner output:
 {"plan":[{"step":"calculate","expression":"879*11"}]}

DEBUG - Parsed plan:
{
  "plan": [
    {
      "step": "calculate",
      "expression": "879*11"
    }
  ]
}

STEP 1: CALCULATE  -> 879*11
Result: 9669

FINAL: 9669

--- Test 2: Translate only ---
DEBUG - Raw planner output:
 {"plan":[{"step":"translate","text":"Good morning, team","target_language":"Hindi"}]}

DEBUG - Parsed plan:
{
  "plan": [
    {
      "step": "translate",
      "text": "Good morning, team",
      "target_language": "Hindi"
    }
  ]
}

STEP 1: TRANSLATE -> 'Good morning, team' to Hindi
Translation: सुप्रभात, टीम

FINAL: सुप्रभात, टीम

--- Test 3: Combined (calculate → translate) ---
DEBUG - Raw planner output:
 {"plan":[{"step":"calculate","expression":"879*11"},{"step":"translate","text":"$LAST_RESULT","target_language":"Hindi"}]}

DEBUG - Parsed plan:
{
  "plan": [
    {
      "step": "calculate",
      "expression": "879*11"
    },
    {
      "step":

'नौ हजार छह सौ उनहत्तर'